### Loans are one of the most crucial aspects of the American economy, but they come with the potential pitfall of loan recipients being unable to pay off these loans resulting in a loss in profit from the stance of the lender. In order to minimize this risk and maximize their ROI, lending institutions in the past have approved loans and assigned risk grades (with corresponding interest rates and term lengths) based on a multitude of factors such as credit score, income, race, and value of collateral. I aim to use modern techniques of data science and machine learning to create a model of classification that would streamline this process and improve upon extant loan grading systems.

### I'm using data provided by NathanGeorge on 1.3M observations of Lending Club (https://www.lendingclub.com/) accepted and declined loan requests from the beginning of 2007 to the end of 2018 with 151 features. (https://www.kaggle.com/wordsforthewise/lending-club)

### Lending Club utilizes a loan risk rating system from A to G, with A having the greatest chance of being paid back in full and also having the lowest interest rate:

![Image of Grades](https://www.moneycrashers.com/wp-content/uploads/2015/04/reward-risk.png)

## Loading libraries

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import time
import lightgbm as lgbm
from pathlib import Path
import pickle
from catboost import CatBoostClassifier, cv, Pool
import scikitplot as skplt
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, tpe, partial

import itertools
from itertools import combinations

import scipy as sp
from scipy.stats import pearsonr, chi2_contingency

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

from datetime import datetime
from dateutil import relativedelta

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

import statsmodels.api as sm 
from statsmodels.graphics.api import abline_plot # For visualling evaluating predictions.
from statsmodels.stats.proportion import proportion_confint

import warnings # For handling error messages.
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

import sklearn.metrics as met
from sklearn import linear_model, preprocessing, model_selection, svm, datasets
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, RFE
from sklearn.linear_model import LassoCV, LogisticRegression, Lasso
from sklearn.metrics import plot_confusion_matrix, auc, confusion_matrix, classification_report, accuracy_score, roc_curve, roc_auc_score, plot_roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale, StandardScaler, LabelEncoder, MinMaxScaler, Binarizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

## Loading dataset

### A preliminary glance of the CSV data showed that there are millions of loans, though the only ones of interest are the accepted loans that were 'Fully Paid' or 'Charged Off'' in the 'loan_status' column.  To save some time, the unique loan 'id' column was set to the index.  According to Lending Club:

"In general, a note goes into Default status when it is 121 or more days past due.  When a note is in Default status, Charge Off occurs no later than 150 days past due (i.e. No later than 30 days after the Default status is reached) when there is no reasonable expectation of sufficient payment to prevent the charge off.  However, bankruptcies may be charged off earlier based on date of bankruptcy notification."

In [2]:
df = pd.read_csv('accepted_2007_to_2018Q4.csv', index_col='id') 

In [3]:
df = df.loc[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

## Exporting dataframe as CSV

In [4]:
df.to_csv( "df1.csv", encoding='utf-8', index=True)